<a href="https://colab.research.google.com/github/vsingh9076/Natural_Language_Processing/blob/master/NER/1_BiLSTM_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credit : [Yosef](https://yoseflaw.medium.com/step-by-step-ner-model-for-bahasa-indonesia-with-pytorch-and-torchtext-6f94fca08406)

# How to perform sequence labeling with pytorch?

I could not find any tutorial that provides a vanilla implementation of sequence labeling with pytorch and torchtext. The closest one is [this](https://colab.research.google.com/github/bentrevett/pytorch-pos-tagging/blob/master/1%20-%20BiLSTM%20for%20PoS%20Tagging.ipynb) (the basis of this notebook). Most of the other examples include miscellaneous improvements, which are hard to differentiate from the essential components. As I intend to understand the foundation of pytorch for NLP, I decided to strip all those advance techniques and focus on understanding each component. Granted, this implementation is far from perfect (well, it is a *minimum* after all). In fact, I point out, in the end, why starting from scratch can be beneficial from learning point-of-view.


In [2]:
# from google.colab import drive
# drive.mount("/content/gdrive")

# !git clone https://github.com/yoseflaw/nerindo.git

Cloning into 'nerindo'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 69 (delta 28), reused 55 (delta 18), pack-reused 0
Receiving objects: 100% (69/69), 309.32 KiB | 6.31 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [1]:
!pip install torchtext==0.6.0

import time
import torch
from torch import nn
from torch.optim import Adam
from torchtext.data import Field, BucketIterator
from torchtext.datasets import SequenceTaggingDataset
from spacy.lang.id import Indonesian

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.16.0
    Uninstalling torchtext-0.16.0:
      Successfully uninstalled torchtext-0.16.0


Before we go to the implementation, let's observe the dataset. I combined two Indonesian NER datasets ([this](https://github.com/yohanesgultom/nlp-experiments) and [this](https://github.com/yusufsyaifudin/indonesia-ner)). I skip the data transformation step in this notebook. The input format that is supported by torchtext for sequence tagging is similar to [CoNLL-2003](https://www.aclweb.org/anthology/W03-0419.pdf). An example is provided below. Each line consists of multiple `columns` separated by `\t` (tab). For simplicity, I keep only the words and the tags. Note that this format assumed tokenized words. Every sentence is separated by a blank line and there is no information on which sentences belong to the same document (all assumed as independent sentences).

To accommodate multi-word entities, the tags follow what is called `BILOU` format: **B**eginning, **I**nside, **L**ast, **O**uter, **U**nit. This is indicated by the character preceding the dash symbol. The type of the named entity is shown by the remaining part. For instance, *Universtas Gadjah Mada* is an `ORGANIZATION` and *Arie Sudjito* is a `PERSON`.

You can check out the datasets [here](https://github.com/yoseflaw/nerindo/tree/master/input).



```
Pengamat	O
politik	O
dari	O
Universitas	B-ORGANIZATION
Gadjah	I-ORGANIZATION
Mada	L-ORGANIZATION
,	O
Arie	B-PERSON
Sudjito	L-PERSON
,	O
menilai	O
,	O
keinginan	O
Ketua	O
Umum	O
Partai	B-ORGANIZATION
Golkar	L-ORGANIZATION
Aburizal	B-PERSON
Bakrie	L-PERSON
untuk	O
maju	O
kembali	O
sebagai	O
ketua	O
umum	O
merupakan	O
pemaksaan	O
kehendak	O
.	O

Menurut	O
dia	O
,	O
ada	O
kesan	O
bahwa	O
Aburizal	U-PERSON
menggunakan	O
segala	O
cara	O
untuk	O
memuluskan	O
jalannya	O
kembali	O
menduduki	O
Golkar	U-ORGANIZATION
1	O
.	O
```



## Building blocks

Using the object-oriented approach, there are at least three components of a pytorch model. Of course, this is not something written in stone, but I find this the most logical separation:

1.   Corpus: handling the dataset and batching
2.   Model: holding the BiLSTM architecture and forward pass logic
3.   Trainer: training sequence of the model



## Corpus

I use torchtext for the corpus preparation. [Here](https://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/) is a good introduction to torchtext if you are interested.

In [3]:
class Corpus(object):

  def __init__(self, input_folder, min_word_freq, batch_size):
    # list all the fields
    self.word_field = Field(lower=True)
    self.tag_field = Field(unk_token=None)
    # create dataset using built-in parser from torchtext
    self.train_dataset, self.val_dataset, self.test_dataset = SequenceTaggingDataset.splits(
        path=input_folder,
        train="train.tsv",
        validation="val.tsv",
        test="test.tsv",
        fields=(("word", self.word_field), ("tag", self.tag_field))
    )
    # convert fields to vocabulary list
    self.word_field.build_vocab(self.train_dataset.word, min_freq=min_word_freq)
    self.tag_field.build_vocab(self.train_dataset.tag)
    # create iterator for batch input
    self.train_iter, self.val_iter, self.test_iter = BucketIterator.splits(
        datasets=(self.train_dataset, self.val_dataset, self.test_dataset),
        batch_size=batch_size
    )
    # prepare padding index to be ignored during model training/evaluation
    self.word_pad_idx = self.word_field.vocab.stoi[self.word_field.pad_token]
    self.tag_pad_idx = self.tag_field.vocab.stoi[self.tag_field.pad_token]

In [4]:
corpus = Corpus(
    input_folder="/content/nerindo/input",
    min_word_freq=3,  # any words occurring less than 3 times will be ignored from vocab
    batch_size=64
)
print(f"Train set: {len(corpus.train_dataset)} sentences")
print(f"Val set: {len(corpus.val_dataset)} sentences")
print(f"Test set: {len(corpus.test_dataset)} sentences")

Train set: 3535 sentences
Val set: 470 sentences
Test set: 468 sentences


## Model

In [5]:
class BiLSTM(nn.Module):

  def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, lstm_layers,
               emb_dropout, lstm_dropout, fc_dropout, word_pad_idx):
    super().__init__()
    self.embedding_dim = embedding_dim
    # LAYER 1: Embedding
    self.embedding = nn.Embedding(
        num_embeddings=input_dim,
        embedding_dim=embedding_dim,
        padding_idx=word_pad_idx
    )
    self.emb_dropout = nn.Dropout(emb_dropout)
    # LAYER 2: BiLSTM
    self.lstm = nn.LSTM(
        input_size=embedding_dim,
        hidden_size=hidden_dim,
        num_layers=lstm_layers,
        bidirectional=True,
        dropout=lstm_dropout if lstm_layers > 1 else 0
    )
    # LAYER 3: Fully-connected
    self.fc_dropout = nn.Dropout(fc_dropout)
    self.fc = nn.Linear(hidden_dim * 2, output_dim)  # times 2 for bidirectional

  def forward(self, sentence):
    # sentence = [sentence length, batch size]
    # embedding_out = [sentence length, batch size, embedding dim]
    embedding_out = self.emb_dropout(self.embedding(sentence))
    # lstm_out = [sentence length, batch size, hidden dim * 2]
    lstm_out, _ = self.lstm(embedding_out)
    # ner_out = [sentence length, batch size, output dim]
    ner_out = self.fc(self.fc_dropout(lstm_out))
    return ner_out

  def init_weights(self):
    # to initialize all parameters from normal distribution
    # helps with converging during training
    for name, param in self.named_parameters():
      nn.init.normal_(param.data, mean=0, std=0.1)

  def init_embeddings(self, word_pad_idx):
    # initialize embedding for padding as zero
    self.embedding.weight.data[word_pad_idx] = torch.zeros(self.embedding_dim)

  def count_parameters(self):
    return sum(p.numel() for p in self.parameters() if p.requires_grad)

In [6]:
bilstm = BiLSTM(
    input_dim=len(corpus.word_field.vocab),
    embedding_dim=300,
    hidden_dim=64,
    output_dim=len(corpus.tag_field.vocab),
    lstm_layers=2,
    emb_dropout=0.5,
    lstm_dropout=0.1,
    fc_dropout=0.25,
    word_pad_idx=corpus.word_pad_idx
)
bilstm.init_weights()
bilstm.init_embeddings(word_pad_idx=corpus.word_pad_idx)
print(f"The model has {bilstm.count_parameters():,} trainable parameters.")
print(bilstm)

The model has 1,383,658 trainable parameters.
BiLSTM(
  (embedding): Embedding(3647, 300, padding_idx=1)
  (emb_dropout): Dropout(p=0.5, inplace=False)
  (lstm): LSTM(300, 64, num_layers=2, dropout=0.1, bidirectional=True)
  (fc_dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=128, out_features=22, bias=True)
)


## Trainer

Here, I include both the training sequence and also the inference function.

In [7]:
class NER(object):

  def __init__(self, model, data, optimizer_cls, loss_fn_cls):
    self.model = model
    self.data = data
    self.optimizer = optimizer_cls(model.parameters())
    self.loss_fn = loss_fn_cls(ignore_index=self.data.tag_pad_idx)

  @staticmethod
  def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

  def accuracy(self, preds, y):
    max_preds = preds.argmax(dim=1, keepdim=True)  # get the index of the max probability
    non_pad_elements = (y != self.data.tag_pad_idx).nonzero()  # prepare masking for paddings
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]])

  def epoch(self):
      epoch_loss = 0
      epoch_acc = 0
      self.model.train()
      for batch in self.data.train_iter:
        # text = [sent len, batch size]
        text = batch.word
        # tags = [sent len, batch size]
        true_tags = batch.tag
        self.optimizer.zero_grad()
        pred_tags = self.model(text)
        # to calculate the loss and accuracy, we flatten both prediction and true tags
        # flatten pred_tags to [sent len, batch size, output dim]
        pred_tags = pred_tags.view(-1, pred_tags.shape[-1])
        # flatten true_tags to [sent len * batch size]
        true_tags = true_tags.view(-1)
        batch_loss = self.loss_fn(pred_tags, true_tags)
        batch_acc = self.accuracy(pred_tags, true_tags)
        batch_loss.backward()
        self.optimizer.step()
        epoch_loss += batch_loss.item()
        epoch_acc += batch_acc.item()
      return epoch_loss / len(self.data.train_iter), epoch_acc / len(self.data.train_iter)

  def evaluate(self, iterator):
      epoch_loss = 0
      epoch_acc = 0
      self.model.eval()
      with torch.no_grad():
          # similar to epoch() but model is in evaluation mode and no backprop
          for batch in iterator:
              text = batch.word
              true_tags = batch.tag
              pred_tags = self.model(text)
              pred_tags = pred_tags.view(-1, pred_tags.shape[-1])
              true_tags = true_tags.view(-1)
              batch_loss = self.loss_fn(pred_tags, true_tags)
              batch_acc = self.accuracy(pred_tags, true_tags)
              epoch_loss += batch_loss.item()
              epoch_acc += batch_acc.item()
      return epoch_loss / len(iterator), epoch_acc / len(iterator)

  # main training sequence
  def train(self, n_epochs):
    for epoch in range(n_epochs):
        start_time = time.time()
        train_loss, train_acc = self.epoch()
        end_time = time.time()
        epoch_mins, epoch_secs = NER.epoch_time(start_time, end_time)
        print(f"Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
        print(f"\tTrn Loss: {train_loss:.3f} | Trn Acc: {train_acc * 100:.2f}%")
        val_loss, val_acc = self.evaluate(self.data.val_iter)
        print(f"\tVal Loss: {val_loss:.3f} | Val Acc: {val_acc * 100:.2f}%")
    test_loss, test_acc = self.evaluate(self.data.test_iter)
    print(f"Test Loss: {test_loss:.3f} |  Test Acc: {test_acc * 100:.2f}%")

  def infer(self, sentence, true_tags=None):
    self.model.eval()
    # tokenize sentence
    nlp = Indonesian()
    tokens = [token.text.lower() for token in nlp(sentence)]
    # transform to indices based on corpus vocab
    numericalized_tokens = [self.data.word_field.vocab.stoi[t] for t in tokens]
    # find unknown words
    unk_idx = self.data.word_field.vocab.stoi[self.data.word_field.unk_token]
    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    # begin prediction
    token_tensor = torch.LongTensor(numericalized_tokens)
    token_tensor = token_tensor.unsqueeze(-1)
    predictions = self.model(token_tensor)
    # convert results to tags
    top_predictions = predictions.argmax(-1)
    predicted_tags = [self.data.tag_field.vocab.itos[t.item()] for t in top_predictions]
    # print inferred tags
    max_len_token = max([len(token) for token in tokens] + [len("word")])
    max_len_tag = max([len(tag) for tag in predicted_tags] + [len("pred")])
    print(
        f"{'word'.ljust(max_len_token)}\t{'unk'.ljust(max_len_token)}\t{'pred tag'.ljust(max_len_tag)}"
        + ("\ttrue tag" if true_tags else "")
        )
    for i, token in enumerate(tokens):
      is_unk = "✓" if token in unks else ""
      print(
          f"{token.ljust(max_len_token)}\t{is_unk.ljust(max_len_token)}\t{predicted_tags[i].ljust(max_len_tag)}"
          + (f"\t{true_tags[i]}" if true_tags else "")
          )
    return tokens, predicted_tags, unks

In [8]:
# this will continue training if the model has been trained before.
# to restart training, run the bilstm creation cell (2 cells above) once again.
ner = NER(
  model=bilstm,
  data=corpus,
  optimizer_cls=Adam,
  loss_fn_cls=nn.CrossEntropyLoss
)
ner.train(10)

Epoch: 01 | Epoch Time: 0m 13s
	Trn Loss: 1.295 | Trn Acc: 80.68%
	Val Loss: 0.809 | Val Acc: 85.67%
Epoch: 02 | Epoch Time: 0m 14s
	Trn Loss: 0.875 | Trn Acc: 83.09%
	Val Loss: 0.653 | Val Acc: 85.67%
Epoch: 03 | Epoch Time: 0m 13s
	Trn Loss: 0.672 | Trn Acc: 83.35%
	Val Loss: 0.527 | Val Acc: 85.65%
Epoch: 04 | Epoch Time: 0m 13s
	Trn Loss: 0.535 | Trn Acc: 85.42%
	Val Loss: 0.442 | Val Acc: 88.06%
Epoch: 05 | Epoch Time: 0m 15s
	Trn Loss: 0.419 | Trn Acc: 88.67%
	Val Loss: 0.358 | Val Acc: 89.97%
Epoch: 06 | Epoch Time: 0m 13s
	Trn Loss: 0.325 | Trn Acc: 90.96%
	Val Loss: 0.323 | Val Acc: 90.78%
Epoch: 07 | Epoch Time: 0m 13s
	Trn Loss: 0.263 | Trn Acc: 92.66%
	Val Loss: 0.328 | Val Acc: 90.45%
Epoch: 08 | Epoch Time: 0m 12s
	Trn Loss: 0.225 | Trn Acc: 93.72%
	Val Loss: 0.309 | Val Acc: 90.69%
Epoch: 09 | Epoch Time: 0m 14s
	Trn Loss: 0.194 | Trn Acc: 94.41%
	Val Loss: 0.300 | Val Acc: 91.60%
Epoch: 10 | Epoch Time: 0m 15s
	Trn Loss: 0.169 | Trn Acc: 95.12%
	Val Loss: 0.319 | Val Ac

In [9]:
# https://regional.kompas.com/read/2020/07/12/15554711/diduga-terlibat-perselingkuhan-ketua-kpu-sumba-barat-diberhentikan
sentence = "\"Menjatuhkan sanksi pemberhentian tetap kepada teradu Sophia Marlinda Djami selaku Ketua KPU Kabupaten Sumba Barat, sejak dibacakannya putusan ini\", ucap Alfitra dalam sidang putusan, Rabu (8/7/2020)."
tags = ["O", "O", "O", "O", "O", "O", "O", "B-PERSON", "I-PERSON", "L-PERSON", "O", "O", "B-ORGANIZATION", "I-ORGANIZATION", "I-ORGANIZATION", "L-ORGANIZATION", "O", "O", "O", "O", "O", "O", "O", "O", "U-PERSON", "O", "O", "O", "O", "B-TIME", "I-TIME", "I-TIME", "I-TIME", "I-TIME", "I-TIME", "I-TIME", "L-TIME", "O"]
words, infer_tags, unknown_tokens = ner.infer(sentence=sentence, true_tags=tags)

word         	unk          	pred tag      	true tag
"            	             	O             	O
menjatuhkan  	             	O             	O
sanksi       	             	O             	O
pemberhentian	✓            	O             	O
tetap        	             	O             	O
kepada       	             	O             	O
teradu       	✓            	O             	O
sophia       	✓            	O             	B-PERSON
marlinda     	✓            	O             	I-PERSON
djami        	✓            	O             	L-PERSON
selaku       	             	O             	O
ketua        	             	O             	O
kpu          	             	U-ORGANIZATION	B-ORGANIZATION
kabupaten    	             	O             	I-ORGANIZATION
sumba        	✓            	B-PERSON      	I-ORGANIZATION
barat        	             	O             	L-ORGANIZATION
,            	             	O             	O
sejak        	             	O             	O
dibacakannya 	✓            	O             	O
putusan      	      

In [10]:
# https://regional.kompas.com/read/2020/07/15/16583081/banjir-bandang-di-masamba-19-korban-meninggal-23-hilang-15000-mengungsi
sentence = "Sementara itu, Kepala Pelaksana BPBD Luwu Utara Muslim Muchtar mengatakan, terdapat 15.000 jiwa mengungsi akibat banjir bandang."
tags = ["O", "O", "O", "O", "O", "B-ORGANIZATION", "I-ORGANIZATION", "L-ORGANIZATION", "B-PERSON", "L-PERSON", "O", "O", "O", "U-QUANTITY", "O", "O", "O", "O", "O", "O"]
words, infer_tags, unknown_tokens = ner.infer(sentence=sentence, true_tags=tags)

word      	unk       	pred tag      	true tag
sementara 	          	O             	O
itu       	          	O             	O
,         	          	O             	O
kepala    	          	O             	O
pelaksana 	✓         	O             	O
bpbd      	✓         	O             	B-ORGANIZATION
luwu      	✓         	I-ORGANIZATION	I-ORGANIZATION
utara     	          	I-PERSON      	L-ORGANIZATION
muslim    	✓         	I-PERSON      	B-PERSON
muchtar   	✓         	L-PERSON      	L-PERSON
mengatakan	          	O             	O
,         	          	O             	O
terdapat  	          	O             	O
15.000    	✓         	B-QUANTITY    	U-QUANTITY
jiwa      	          	I-QUANTITY    	O
mengungsi 	✓         	L-QUANTITY    	O
akibat    	          	O             	O
banjir    	          	O             	O
bandang   	✓         	O             	O
.         	          	O             	O


## Now what?

As you can see from the result, it is far from perfect and tends to overfit. Coming up with a list of possible improvements is easy, but deciding which one is the key. Should we incorporate pre-trained embeddings? Is accuracy the best metric to measure the performance? How about character-level features? Maybe include an attention layer?

We often overlook this step while following an example/a tutorial. We carelessly jump to the model that includes everything. Tinkering with those choices helps to understand the effect of each improvement. Which method should we prioritize to implement?

Continue here: [Part 2: Word2Vec](https://colab.research.google.com/drive/14i8ixDeJv6YIebvW6aKGFOJyXTUCTfR9?usp=sharing).
